In [ ]:
import numpy as np

In [ ]:
import os
assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

## 棋譜データの読み込み

In [ ]:
"!mkdir drive/MyDrive/colab
"!mkdir drive/MyDrive/colab/ai_reversi

In [ ]:
!ls drive/MyDrive/colab/ai_reversi

kihuFixed.txt


In [ ]:
#with open("drive/MyDrive/colab/ai_reversi/kihuFixed.txt") as f:
#  lines = f.read().split("\n")

In [ ]:
len(lines)

6066496

## データの前処理

In [ ]:
def generate_next_flipped(board, row, col):
  flipped = []
  for dx in range(-1,2):
    for dy in range(-1,2):
      if dx == 0 and dy == 0:
        continue
      flipped_tmp = []
      for k in range(1,8):
        x = row + dx*k
        y = col + dy*k
        if not (0<=x<8 and 0<=y<8):
          flipped_tmp = []
          break
        if board[x*8+y] == 0:
          flipped_tmp = []
          break
        if board[x*8+y] == 1:
          flipped += flipped_tmp
          break
        if board[x*8+y] == 2:
          flipped_tmp.append((x,y))

  if not flipped:
    #print(np.array(board).reshape((8,8)))
    #print(row,col)
    #assert False

    # パス
    return -1

  board[row*8+col] = 1
  for x,y in flipped:
    board[x*8+y] = 1

  #print(board.reshape((8,8)))

  for i in range(8):
    for j in range(8):
      if board[i*8+j] == 1:
        board[i*8+j] = 2
      elif board[i*8+j] == 2:
        board[i*8+j] = 1

  return board

In [ ]:
board = np.zeros((64,),int)
board[3*8+3] = board[4*8+4] = 1
board[3*8+4] = board[4*8+3] = 2
row, col = 3,5

In [ ]:
generate_next_flipped(board.copy(), row, col).reshape((8,8))

[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 1 1 1 0 0]
 [0 0 0 2 1 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]


array([[0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 2, 2, 2, 0, 0],
       [0, 0, 0, 1, 2, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0]])

In [ ]:
def find_next_action(before,after,row,col):
  for i in range(8):
    for j in range(8):
      if before[i*8+j]==0 and after[i*8+j] != 0 and (i,j) != (row,col):
        return (i,j)

  #print(np.array(before).reshape((8,8)))
  #print(np.array(after).reshape((8,8)))
  #print(row,col)
  #assert False

  # パス
  return -1,-1

In [ ]:
find_next_action(board.copy(), )

In [ ]:
for i in range(10):
  board = draw(X[i,:,:,0],X[i,:,:,1])
  print(board)

array([[False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False],
       [False, False, False, False,  True, False, False, False],
       [False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False]])

In [ ]:
def preprocess():
  tmp1,tmp2 = [],[]
  with open("drive/MyDrive/colab/ai_reversi/kihuFixed.txt") as f:
    while line:=f.readline():
      if len(tmp1) % 1000000 == 0:
        print(f"processing {len(tmp1)}-th...")
      line = line.strip().split()
      if not line:
        continue
      t = list(map(int,line[:64]))
      col,row = map(int,line[-4:-2])
      row -= 1
      col -= 1

      if len(tmp1):
        before = tmp1[-1][:]
        row0,col0 = tmp2[-1]
        after = generate_next_flipped(before[:], row0, col0)

        # パスではない
        if after != -1:
          row2,col2 = find_next_action(before, t, row0, col0)
          tmp1.append(after)
          tmp2.append([row2,col2])

      tmp1.append(t)
      tmp2.append([row,col])

  n_sample = len(tmp1)

  tmp1 = np.array(tmp1, dtype=np.int8).reshape(n_sample,8,8)
  tmp2 = np.array(tmp2, dtype=np.int8)

  #print(tmp1.shape, tmp2.shape)

  X = np.stack([(tmp1==1),(tmp1==2)],axis=0).transpose(1,2,3,0).astype(bool)

  #print(X.shape)

  y = np.zeros(shape=(n_sample,8,8), dtype=bool)
  y[np.arange(n_sample),tmp2[:,0],tmp2[:,1]] = 1

  return X,y

In [ ]:
X,y = preprocess()

processing 0-th...
processing 1000000-th...
processing 2000000-th...
processing 3000000-th...
processing 5000000-th...
processing 6000000-th...
processing 7000000-th...
processing 9000000-th...


In [ ]:
def augument_data(data,axes=(0,1)):
  data90 = np.rot90(data,k=1,axes=axes)
  data180 = np.rot90(data,k=2,axes=axes)
  data270 = np.rot90(data,k=3,axes=axes)
  return np.vstack([data,data90,data180,data270])

In [ ]:
X2 = augument_data(X, axes=(1,2))
y2 = augument_data(y, axes=(1,2))

In [ ]:
def draw(x1,x2):
  ret = np.zeros((8,8),int)
  ret[x1] = 1
  ret[x2] = 2
  #print(ret)
  return ret

In [ ]:
for i in range(10):
  print(draw(X[i,:,:,0],X[i,:,:,1]))
  print(y[i].astype(int))
  print()

[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 2 2 2 0 0 0]
 [0 0 0 2 1 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]
[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]

[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 2 0 0 0]
 [0 0 1 1 2 0 0 0]
 [0 0 0 1 2 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]
[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]

[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 1 2 0 0]
 [0 0 2 2 2 0 0 0]
 [0 0 0 2 1 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]
[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]

[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 2 1 0 0]
 [0 0 1 2 1 0 0 0]
 [0

In [ ]:
X[1,:,:,0]

array([[False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False],
       [False, False, False, False,  True, False, False, False],
       [False, False, False, False, False, False, False, False],
       [False, False, False, False,  True, False, False, False],
       [False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False]])

In [ ]:
y[1]

array([[False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False],
       [False, False, False, False, False,  True, False, False],
       [False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False]])

In [ ]:
X2.shape

(37807816, 8, 8, 2)

In [ ]:
np.save("drive/MyDrive/colab/ai_reversi/state.npy", X2)
np.save("drive/MyDrive/colab/ai_reversi/action.npy", y2)

## 学習

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

In [ ]:
from tensorflow.keras import layers

def block(out_channels, ksize, pad=1):
    return tf.keras.Sequential([
        layers.Conv2D(out_channels, ksize, padding='same' if pad == 1 else 'valid', activation='relu')
    ])

def sl_policy():
    ksize = 3
    inputs = layers.Input(shape=(8, 8, 2))
    h = block(64, ksize)(inputs)
    h = block(128, ksize)(h)
    h = block(128, ksize)(h)
    h = block(128, ksize)(h)
    h = block(128, ksize)(h)
    h = block(128, ksize)(h)
    h = block(128, ksize)(h)
    h = block(128, ksize)(h)
    h = layers.Conv2D(1, 1, use_bias=False)(h)
    h = layers.Flatten()(h)  # 2次元の出力を1次元に平坦化
    outputs = layers.Dense(64)(h)  # 出力層のユニット数を64に設定し、活性化関数は指定しない

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import os

# TPUの設定
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

# Google Driveのマウント
#from google.colab import drive
#drive.mount('/content/drive')

Running on TPU  grpc://10.91.97.178:8470


REPLICAS:  8


In [ ]:
# 1エポックの中で細かくチェックポイントを作るためのコールバック

import os
import tensorflow as tf

class CustomCheckpoint(tf.keras.callbacks.Callback):
    def __init__(self, filepath, steps_per_epoch, save_freq):
        super(CustomCheckpoint, self).__init__()
        self.filepath = filepath
        self.steps_per_epoch = steps_per_epoch
        self.save_freq = save_freq
        self.batch_counter = 0

    def on_epoch_begin(self, epoch, logs=None):
        self.batch_counter = 0  # エポックの開始時にバッチカウンターをリセット

    def on_batch_end(self, batch, logs=None):
        self.batch_counter += 1
        if self.batch_counter % self.save_freq == 0:
            # チェックポイントのファイルパスを設定
            filepath = self.filepath.format(epoch=self.epoch + 1, batch=batch + 1)
            self.model.save_weights(filepath, overwrite=True)
            print(f'\nSaved checkpoint to: {filepath}')

In [ ]:
X = np.load('drive/MyDrive/colab/ai_reversi/state.npy')
y = np.load('drive/MyDrive/colab/ai_reversi/action.npy')

In [ ]:
np.isnan(y).sum()

0

In [ ]:
(y==np.inf).sum()

0

In [ ]:
from datetime import datetime
import pickle

In [ ]:
# チェックポイントのコールバック
#checkpoint_path = "/content/drive/MyDrive/colab/ai_reversi/reversi_model/cp-{epoch:04d}.ckpt"


# 学習データの読み込み
#X_train = np.load('drive/MyDrive/colab/ai_reversi/state.npy')
#y_train = np.load('drive/MyDrive/colab/ai_reversi/action.npy')

# モデルの作成とコンパイル
"""
with strategy.scope():
    #model = sl_policy()
    with open("/content/drive/MyDrive/colab/ai_reversi/reversi_model/cp-2024-03-0821:15:05.760753.pickle", "rb") as f:
      model = pickle.load(f)
      model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
"""

#model = tf.keras.models.load_model("drive/MyDrive/colab/ai_reversi/reversi_model/cp-2024-03-08 11:52:25.652748.ckpt.data-00000-of-00001.h5")

#batch_size = 64
#n = len(X_train)

# split dataset because large-size data cause serializing error at TPU
n = 37807816
n_batch = 16

for j in range(3):
    for i in range(n_batch):

      if j == 0 and i == 0:
        continue

      if j  == 0 and i <= 11:
        continue

      print(f"Training the {i+1}-th block, the {j+1}-th iteration of the data...")

      X_train2 = np.load('drive/MyDrive/colab/ai_reversi/state.npy')[n//n_batch*i:min(n//n_batch*(i+1),n)].astype(float)
      y_train2 = np.load('drive/MyDrive/colab/ai_reversi/action.npy')[n//n_batch*i:min(n//n_batch*(i+1),n)]
      #y_train2 = tf.keras.utils.to_categorical(y_train2, num_classes=64)
      y_train2 = np.array([np.argmax(one_hot.flatten()) for one_hot in y_train2])

      #del X_train,y_train

      # コールバックのインスタンス化
      #steps_per_epoch = len(X_train2) // batch_size
      #save_freq = steps_per_epoch // 10  # 1エポック内で10回保存する

      #checkpoint_path = "/content/drive/MyDrive/colab/ai_reversi/reversi_model/cp-{i:04}-{epoch:04d}-{batch:04d}.ckpt"
      #cp_callback = CustomCheckpoint(filepath=checkpoint_path, steps_per_epoch=steps_per_epoch, save_freq=save_freq)

      checkpoint_path = f"/content/drive/MyDrive/colab/ai_reversi/reversi_model/cp-{i:04d}"
      checkpoint_path += "-{epoch:04d}.ckpt"
      checkpoint_dir = os.path.dirname(checkpoint_path)
      cp_callback = tf.keras.callbacks.ModelCheckpoint(
          filepath=checkpoint_path,
          verbose=1,
          save_weights_only=True)

      model_path = f"/content/drive/MyDrive/colab/ai_reversi/reversi_model/cp-{str(datetime.now()).replace(' ','')}"

      # 学習
      #model.fit(X_train2, y_train2, epochs=1, callbacks=[cp_callback])
      model.fit(X_train2, y_train2, epochs=1)

      checkpoint = tf.train.Checkpoint(model=model)
      local_device_option = tf.train.CheckpointOptions(experimental_io_device="/job:localhost")
      checkpoint.write(
          model_path + ".ckpt",
          options=local_device_option)

      with open(model_path + ".pickle", "wb") as f:
          pickle.dump(model, f)

Training the 13-th block, the 1-th iteration of the data...
73844/73844 [==============================] - 945s 13ms/step - loss: 1.4268 - accuracy: 0.5199
Training the 14-th block, the 1-th iteration of the data...
73844/73844 [==============================] - 933s 13ms/step - loss: 1.4072 - accuracy: 0.5231
Training the 15-th block, the 1-th iteration of the data...
73844/73844 [==============================] - 881s 12ms/step - loss: 1.3070 - accuracy: 0.5516
Training the 16-th block, the 1-th iteration of the data...
73844/73844 [==============================] - 913s 12ms/step - loss: 1.3389 - accuracy: 0.5432
Training the 1-th block, the 2-th iteration of the data...
73844/73844 [==============================] - 930s 13ms/step - loss: 1.4194 - accuracy: 0.5220
Training the 2-th block, the 2-th iteration of the data...
73844/73844 [==============================] - 906s 12ms/step - loss: 1.4006 - accuracy: 0.5253
Training the 3-th block, the 2-th iteration of the data...
73844/7

Training the 2-th block of the data...
WARNING:tensorflow:`period` argument is deprecated. Please use `save_freq` to specify the frequency in number of batches seen.
73844/73844 [==============================] - 918s 12ms/step - loss: 1.6377 - accuracy: 0.4607
Training the 3-th block of the data...
WARNING:tensorflow:`period` argument is deprecated. Please use `save_freq` to specify the frequency in number of batches seen.
73844/73844 [==============================] - 933s 13ms/step - loss: 1.5102 - accuracy: 0.4959
Training the 4-th block of the data...
WARNING:tensorflow:`period` argument is deprecated. Please use `save_freq` to specify the frequency in number of batches seen.
73844/73844 [==============================] - 918s 12ms/step - loss: 1.5202 - accuracy: 0.4932
Training the 5-th block of the data...
WARNING:tensorflow:`period` argument is deprecated. Please use `save_freq` to specify the frequency in number of batches seen.
73844/73844 [==============================] - 919s 12ms/step - loss: 1.6181 - accuracy: 0.4668
Training the 6-th block of the data...
WARNING:tensorflow:`period` argument is deprecated. Please use `save_freq` to specify the frequency in number of batches seen.
73844/73844 [==============================] - 941s 13ms/step - loss: 1.5588 - accuracy: 0.4814
Training the 7-th block of the data...
WARNING:tensorflow:`period` argument is deprecated. Please use `save_freq` to specify the frequency in number of batches seen.
73844/73844 [==============================] - 922s 12ms/step - loss: 1.4457 - accuracy: 0.5129
Training the 8-th block of the data...
WARNING:tensorflow:`period` argument is deprecated. Please use `save_freq` to specify the frequency in number of batches seen.
73844/73844 [==============================] - 928s 13ms/step - loss: 1.4588 - accuracy: 0.5095
Training the 9-th block of the data...
WARNING:tensorflow:`period` argument is deprecated. Please use `save_freq` to specify the frequency in number of batches seen.
73844/73844 [==============================] - 1125s 15ms/step - loss: 1.5499 - accuracy: 0.4849
Training the 10-th block of the data...
WARNING:tensorflow:`period` argument is deprecated. Please use `save_freq` to specify the frequency in number of batches seen.
73844/73844 [==============================] - 946s 13ms/step - loss: 1.5054 - accuracy: 0.4955
Training the 11-th block of the data...
WARNING:tensorflow:`period` argument is deprecated. Please use `save_freq` to specify the frequency in number of batches seen.
73844/73844 [==============================] - 940s 13ms/step - loss: 1.3985 - accuracy: 0.5253
Training the 12-th block of the data...
WARNING:tensorflow:`period` argument is deprecated. Please use `save_freq` to specify the frequency in number of batches seen.
73844/73844 [==============================] - 972s 13ms/step - loss: 1.4192 - accuracy: 0.5202
Training the 13-th block of the data...
WARNING:tensorflow:`period` argument is deprecated. Please use `save_freq` to specify the frequency in number of batches seen.
73844/73844 [==============================] - 925s 13ms/step - loss: 1.5140 - accuracy: 0.4946
Training the 14-th block of the data...
WARNING:tensorflow:`period` argument is deprecated. Please use `save_freq` to specify the frequency in number of batches seen.
73844/73844 [==============================] - 927s 13ms/step - loss: 1.4755 - accuracy: 0.5036
Training the 15-th block of the data...
WARNING:tensorflow:`period` argument is deprecated. Please use `save_freq` to specify the frequency in number of batches seen.
73844/73844 [==============================] - 968s 13ms/step - loss: 1.3727 - accuracy: 0.5325
Training the 16-th block of the data...
WARNING:tensorflow:`period` argument is deprecated. Please use `save_freq` to specify the frequency in number of batches seen.
73844/73844 [==============================] - 991s 13ms/step - loss: 1.3949 - accuracy: 0.5267Training the 2-th block of the data...
WARNING:tensorflow:`period` argument is deprecated. Please use `save_freq` to specify the frequency in number of batches seen.
73844/73844 [==============================] - 918s 12ms/step - loss: 1.6377 - accuracy: 0.4607
Training the 3-th block of the data...
WARNING:tensorflow:`period` argument is deprecated. Please use `save_freq` to specify the frequency in number of batches seen.
73844/73844 [==============================] - 933s 13ms/step - loss: 1.5102 - accuracy: 0.4959
Training the 4-th block of the data...
WARNING:tensorflow:`period` argument is deprecated. Please use `save_freq` to specify the frequency in number of batches seen.
73844/73844 [==============================] - 918s 12ms/step - loss: 1.5202 - accuracy: 0.4932
Training the 5-th block of the data...
WARNING:tensorflow:`period` argument is deprecated. Please use `save_freq` to specify the frequency in number of batches seen.
73844/73844 [==============================] - 919s 12ms/step - loss: 1.6181 - accuracy: 0.4668
Training the 6-th block of the data...
WARNING:tensorflow:`period` argument is deprecated. Please use `save_freq` to specify the frequency in number of batches seen.
73844/73844 [==============================] - 941s 13ms/step - loss: 1.5588 - accuracy: 0.4814
Training the 7-th block of the data...
WARNING:tensorflow:`period` argument is deprecated. Please use `save_freq` to specify the frequency in number of batches seen.
73844/73844 [==============================] - 922s 12ms/step - loss: 1.4457 - accuracy: 0.5129
Training the 8-th block of the data...
WARNING:tensorflow:`period` argument is deprecated. Please use `save_freq` to specify the frequency in number of batches seen.
73844/73844 [==============================] - 928s 13ms/step - loss: 1.4588 - accuracy: 0.5095
Training the 9-th block of the data...
WARNING:tensorflow:`period` argument is deprecated. Please use `save_freq` to specify the frequency in number of batches seen.
73844/73844 [==============================] - 1125s 15ms/step - loss: 1.5499 - accuracy: 0.4849
Training the 10-th block of the data...
WARNING:tensorflow:`period` argument is deprecated. Please use `save_freq` to specify the frequency in number of batches seen.
73844/73844 [==============================] - 946s 13ms/step - loss: 1.5054 - accuracy: 0.4955
Training the 11-th block of the data...
WARNING:tensorflow:`period` argument is deprecated. Please use `save_freq` to specify the frequency in number of batches seen.
73844/73844 [==============================] - 940s 13ms/step - loss: 1.3985 - accuracy: 0.5253
Training the 12-th block of the data...
WARNING:tensorflow:`period` argument is deprecated. Please use `save_freq` to specify the frequency in number of batches seen.
73844/73844 [==============================] - 972s 13ms/step - loss: 1.4192 - accuracy: 0.5202
Training the 13-th block of the data...
WARNING:tensorflow:`period` argument is deprecated. Please use `save_freq` to specify the frequency in number of batches seen.
73844/73844 [==============================] - 925s 13ms/step - loss: 1.5140 - accuracy: 0.4946
Training the 14-th block of the data...
WARNING:tensorflow:`period` argument is deprecated. Please use `save_freq` to specify the frequency in number of batches seen.
73844/73844 [==============================] - 927s 13ms/step - loss: 1.4755 - accuracy: 0.5036
Training the 15-th block of the data...
WARNING:tensorflow:`period` argument is deprecated. Please use `save_freq` to specify the frequency in number of batches seen.
73844/73844 [==============================] - 968s 13ms/step - loss: 1.3727 - accuracy: 0.5325
Training the 16-th block of the data...
WARNING:tensorflow:`period` argument is deprecated. Please use `save_freq` to specify the frequency in number of batches seen.
73844/73844 [==============================] - 991s 13ms/step - loss: 1.3949 - accuracy: 0.5267

Training the 1-th block, the 1-th iteration of the data...
73844/73844 [==============================] - 1046s 14ms/step - loss: 1.4816 - accuracy: 0.5038

Training the 2-th block, the 1-th iteration of the data...
73844/73844 [==============================] - 972s 13ms/step - loss: 1.4500 - accuracy: 0.5109
Training the 3-th block, the 1-th iteration of the data...
73844/73844 [==============================] - 990s 13ms/step - loss: 1.3484 - accuracy: 0.5389
Training the 4-th block, the 1-th iteration of the data...
73844/73844 [==============================] - 999s 14ms/step - loss: 1.3741 - accuracy: 0.5321
Training the 5-th block, the 1-th iteration of the data...
73844/73844 [==============================] - 990s 13ms/step - loss: 1.4609 - accuracy: 0.5098
Training the 6-th block, the 1-th iteration of the data...
73844/73844 [==============================] - 998s 14ms/step - loss: 1.4333 - accuracy: 0.5155
Training the 7-th block, the 1-th iteration of the data...
73844/73844 [==============================] - 1019s 14ms/step - loss: 1.3323 - accuracy: 0.5439
Training the 8-th block, the 1-th iteration of the data...
55346/73844 [=====================>........] - ETA: 4:14 - loss: 1.3646 - accuracy: 0.5349

		Training the 8-th block, the 1-th iteration of the data...
		73844/73844 [==============================] - 884s 12ms/step - loss: 1.3595 - accuracy: 0.5363
		Training the 9-th block, the 1-th iteration of the data...
		73844/73844 [==============================] - 878s 12ms/step - loss: 1.4436 - accuracy: 0.5147
		Training the 10-th block, the 1-th iteration of the data...
		73844/73844 [==============================] - 876s 12ms/step - loss: 1.4188 - accuracy: 0.5197
		Training the 11-th block, the 1-th iteration of the data...
		73844/73844 [==============================] - 887s 12ms/step - loss: 1.3179 - accuracy: 0.5481
		Training the 12-th block, the 1-th iteration of the data...

Training the 13-th block, the 1-th iteration of the data...
73844/73844 [==============================] - 945s 13ms/step - loss: 1.4268 - accuracy: 0.5199
Training the 14-th block, the 1-th iteration of the data...
73844/73844 [==============================] - 933s 13ms/step - loss: 1.4072 - accuracy: 0.5231
Training the 15-th block, the 1-th iteration of the data...
73844/73844 [==============================] - 881s 12ms/step - loss: 1.3070 - accuracy: 0.5516
Training the 16-th block, the 1-th iteration of the data...
73844/73844 [==============================] - 913s 12ms/step - loss: 1.3389 - accuracy: 0.5432
Training the 1-th block, the 2-th iteration of the data...
73844/73844 [==============================] - 930s 13ms/step - loss: 1.4194 - accuracy: 0.5220
Training the 2-th block, the 2-th iteration of the data...
73844/73844 [==============================] - 906s 12ms/step - loss: 1.4006 - accuracy: 0.5253
Training the 3-th block, the 2-th iteration of the data...
73844/73844 [==============================] - 886s 12ms/step - loss: 1.3003 - accuracy: 0.5533
Training the 4-th block, the 2-th iteration of the data...
73844/73844 [==============================] - 884s 12ms/step - loss: 1.3331 - accuracy: 0.5444
Training the 5-th block, the 2-th iteration of the data...
73844/73844 [==============================] - 873s 12ms/step - loss: 1.4094 - accuracy: 0.5248
Training the 6-th block, the 2-th iteration of the data...
73844/73844 [==============================] - 837s 11ms/step - loss: 1.3914 - accuracy: 0.5282
Training the 7-th block, the 2-th iteration of the data...
73844/73844 [==============================] - 864s 12ms/step - loss: 1.2898 - accuracy: 0.5564
Training the 8-th block, the 2-th iteration of the data...
73844/73844 [==============================] - 938s 13ms/step - loss: 1.3227 - accuracy: 0.5475
Training the 9-th block, the 2-th iteration of the data...
73844/73844 [==============================] - 919s 12ms/step - loss: 1.4026 - accuracy: 0.5268
Training the 10-th block, the 2-th iteration of the data...
73844/73844 [==============================] - 885s 12ms/step - loss: 1.3862 - accuracy: 0.5297
Training the 11-th block, the 2-th iteration of the data...
73844/73844 [==============================] - 870s 12ms/step - loss: 1.2858 - accuracy: 0.5576
Training the 12-th block, the 2-th iteration of the data...
73844/73844 [==============================] - 865s 12ms/step - loss: 1.3286 - accuracy: 0.5456
Training the 13-th block, the 2-th iteration of the data...
73844/73844 [==============================] - 847s 11ms/step - loss: 1.3970 - accuracy: 0.5284
Training the 14-th block, the 2-th iteration of the data...
73844/73844 [==============================] - 836s 11ms/step - loss: 1.3803 - accuracy: 0.5308
Training the 15-th block, the 2-th iteration of the data...
73844/73844 [==============================] - 832s 11ms/step - loss: 1.2792 - accuracy: 0.5594
Training the 16-th block, the 2-th iteration of the data...
73844/73844 [==============================] - 822s 11ms/step - loss: 1.3137 - accuracy: 0.5504
Training the 1-th block, the 3-th iteration of the data...
73844/73844 [==============================] - 832s 11ms/step - loss: 1.3892 - accuracy: 0.5309
Training the 2-th block, the 3-th iteration of the data...
73844/73844 [==============================] - 817s 11ms/step - loss: 1.3754 - accuracy: 0.5328
Training the 3-th block, the 3-th iteration of the data...
73844/73844 [==============================] - 813s 11ms/step - loss: 1.2728 - accuracy: 0.5615
Training the 4-th block, the 3-th iteration of the data...
73844/73844 [==============================] - 810s 11ms/step - loss: 1.3089 - accuracy: 0.5516
Training the 5-th block, the 3-th iteration of the data...
73844/73844 [==============================] - 808s 11ms/step - loss: 1.3843 - accuracy: 0.5323
Training the 6-th block, the 3-th iteration of the data...
73844/73844 [==============================] - 808s 11ms/step - loss: 1.3707 - accuracy: 0.5343
Training the 7-th block, the 3-th iteration of the data...
73844/73844 [==============================] - 810s 11ms/step - loss: 1.2678 - accuracy: 0.5627
Training the 8-th block, the 3-th iteration of the data...
51442/73844 [===================>..........] - ETA: 4:03 - loss: 1.3104 - accuracy: 0.5512